<a href="https://colab.research.google.com/github/rebeccadimatteo/FitDiary_FIA/blob/main/regressioneLineare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressione lineare multipla

## Ottenimento del dataset

In [17]:
import pandas as pd
import os
from six.moves import urllib

DOWNLOAD_ROOT="https://raw.githubusercontent.com/rebeccadimatteo/FitDiary_FIA/master/dataset.csv"

def fetch_file_data1(file_url1=DOWNLOAD_ROOT, file_path1="file"):

    os.makedirs(file_path1, exist_ok=True)

    csv_path1 = os.path.join(file_path1, "dataset.csv")

    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()

df=pd.read_csv("file/dataset.csv")
df.head(10)

,id,sesso,eta,mese,peso,kcal,pesoStimato
0,1,0,46,1,67.00000,2061,-0.438622
1,1,0,46,2,67.43862,1537,-1.277814
2,1,0,46,2,68.71644,3173,0.399622
3,1,0,46,2,68.31682,2684,0.281297
4,1,0,46,2,68.03552,1568,-1.225765
5,1,0,46,2,69.26129,2064,-0.937984
6,1,0,46,2,70.19928,1768,-1.347285
7,1,0,46,2,71.54656,1679,-1.076831
8,1,0,46,2,72.62339,2262,-0.448276
9,1,0,46,2,73.07166,1695,-1.614159


## 1)DataCleaning

### Descrizione del dataset

In [18]:
df.describe()

,id,sesso,eta,mese,peso,kcal,pesoStimato
count,36000.000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000
mean,500.500,0.481000,38.973000,1.972222,93.960686,2854.008639,-0.577729
std,288.679,0.499646,12.283911,0.164338,16.473259,739.296984,0.931751
min,1.000,0.000000,18.000000,1.000000,58.159405,1476.000000,-3.196676
25%,250.750,0.000000,28.000000,2.000000,80.624093,2294.000000,-1.165279
50%,500.500,0.000000,39.000000,2.000000,93.661348,2725.000000,-0.487886
75%,750.250,1.000000,49.000000,2.000000,106.744870,3285.250000,0.000000
max,1000.000,1.000000,59.000000,2.000000,143.593690,6117.000000,1.402835


### verifica valori null nel dataset

In [19]:
df.isnull().any()

id             False
sesso          False
eta            False
mese           False
peso           False
kcal           False
pesoStimato    False
dtype: bool

### verifica valori N/A nel dataset

In [20]:
df.isna().any()

id             False
sesso          False
eta            False
mese           False
peso           False
kcal           False
pesoStimato    False
dtype: bool

## 2)Divisione in Training e Test Sets

### 2.1)k-fold e Repeated K-fold

In [65]:
from sklearn.model_selection import RepeatedKFold, KFold
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

#Scelta variabile dipendente (y) e indipendenti (X)
X=df.drop('pesoStimato',1)
y=df.pesoStimato

#stratifiedKFold con 10
kf = KFold(n_splits=10,random_state=1, shuffle=True)
rkf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=1)

#z-Score Normalization Scaler
scaler_StandardScaler = StandardScaler()

#MinMax-Normalization Scaler
scaler_MinMaxScaler = MinMaxScaler()

#selector per Feature selection
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))

for train_index,test_index in kf.split(X,y):
   X_train, X_test = X.iloc[train_index], X.iloc[test_index]
   y_train, y_test = y.iloc[train_index], y.iloc[test_index]

   #feture Scaling StandardScaler sui traing test
   X_train_z = scaler_StandardScaler.fit_transform(X_train)

   #feture Scaling MinMax sui traing test
   X_train_mm = scaler_MinMaxScaler.fit_transform(X_train)


   #eliminazione feature con bassa varianza
   X_train_z = sel.fit_transform(X_train_z)

   print(X_train_z)

  

  

[[-1.73392225 -0.96136311  0.57339213 -5.87590978 -1.63407691 -1.0723443 ]
 [-1.73392225 -0.96136311  0.57339213  0.17018641 -1.60748045 -1.78115715]
 [-1.73392225 -0.96136311  0.57339213  0.17018641 -1.52999769  0.43185397]
 ...
 [ 1.73063148  1.0401897   0.73628762  0.17018641 -1.15060007 -0.5637305 ]
 [ 1.73063148  1.0401897   0.73628762  0.17018641 -1.09710421 -0.77745651]
 [ 1.73063148  1.0401897   0.73628762  0.17018641 -1.01661878 -0.94789624]]
[[-1.72975982 -0.96380282  0.57520306 -5.95016759 -1.63913024 -1.07287681]
 [-1.72975982 -0.96380282  0.57520306  0.16806249 -1.61251213 -1.7815223 ]
 [-1.72975982 -0.96380282  0.57520306  0.16806249 -1.55921764 -0.23034601]
 ...
 [ 1.72972732  1.03755662  0.73811022  0.16806249 -1.15525975 -0.5643831 ]
 [ 1.72972732  1.03755662  0.73811022  0.16806249 -1.10172033 -0.77805865]
 [ 1.72972732  1.03755662  0.73811022  0.16806249 -1.02116937 -0.94845814]]
[[-1.73213044 -0.96219554  0.57179285 -5.92964706 -1.63664217 -1.07148872]
 [-1.73213044

## 3)Feature Scaling

### 3.1)StandardScaler (z-score Normalization)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_StandardScaler = StandardScaler()

df[df.columns] = scaler_StandardScaler.fit_transform(df.values)
df

## 4)Feature Selction

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib import style

style.use('seaborn-whitegrid')

plt.rcParams['figure.figsize'] = (20,10)

sb.pairplot(df)

plt.savefig('pairplor.png')